In [11]:
import os, sys
from pathlib import Path
import xarray as xr
import geopandas as gpd

project_root = Path('/home/glen/works/forcing_generate_test/')
basin_id = '04040000'

In [12]:
subbasins_shp = gpd.read_file(project_root / f'GRIP-GL_shapefiles' / 'GRIP_GL_141_calibration_catchment_info.shp')

sel = subbasins_shp.loc[subbasins_shp['Gauge_ID'] == basin_id]
sel = sel.reset_index(drop=True)
subbasins_shp = sel[['SubId', 'INSIDE_Y','INSIDE_X','Gauge_ID', 'geometry']]
subbasins_shp = subbasins_shp.dissolve(by='SubId', aggfunc='mean').reset_index()
subbasins_shp['Gauge_ID'] = basin_id
subbasins_shp.rename(columns={'INSIDE_X':'centroid_x', 'INSIDE_Y':'centroid_y'}, inplace=True)
subbasins_shp.to_file(project_root / f'{basin_id}_subbasins.shp')

basin_shp = subbasins_shp.dissolve(by='Gauge_ID')
basin_shp.to_file(project_root / f'{basin_id}.shp')

In [13]:
os.getcwd()

'/home/glen/works/forcing_generate_test/preprocessing_scripts'

In [17]:
os.system(f"python derive_grid_weights.py\
                    -i ../RDRSv2.1_precip_UTC5_1h.nc\
                    -d 'rlon,rlat' -v 'lon,lat' \
                    -r ../{basin_id}.shp\
                    -o ../grid_weights_{basin_id}.txt -a -c 'SubId'")

 
   (1) Reading NetCDF (grid) data ...
   >>> order of dimensions correct for variable "lon"
   >>> order of dimensions correct for variable "lat"
 
   (2) Reading shapefile data ...
   >>> HRU_IDs found =  [1007.0]   (total:  1 )
 
   (3) Generate shapes for NetCDF grid cells ...
   >>> Latitudes done: 0 of 12
   >>> Latitudes done: 10 of 12
 
   (4) Deriving weights ...
   >>> 1007,2,2,22,0.003803293964323388  (corrected to 0.0038032939643233357)
   >>> 1007,2,3,23,0.011440692798719896  (corrected to 0.011440692798719738)
   >>> 1007,2,4,24,0.00014106857310787718  (corrected to 0.00014106857310787523)
   >>> 1007,2,5,25,2.4149151057657816e-05  (corrected to 2.4149151057657484e-05)
   >>> 1007,3,2,32,0.02078242594385064  (corrected to 0.020782425943850356)
   >>> 1007,3,3,33,0.028132560326822167  (corrected to 0.02813256032682178)
   >>> 1007,3,4,34,0.02628480984595403  (corrected to 0.026284809845953667)
   >>> 1007,3,5,35,0.024779895021807014  (corrected to 0.024779895021806674)
  

0